# Day 18 - 2D cellular automation

Today's puzzle is touching upon [cellular automation](https://en.wikipedia.org/wiki/Cellular_automaton) again, but this time we have a finite 2D grid and 3 states, rather than just on or off.

Because this is a grid, I'll use numpy for this version. We can use [`scipy.signal.convolve2d()`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.signal.convolve2d.html) to give us neighbour counts, and [`numpy.select()`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.select.html) to create the next step output grid.

Given a 3 x 3 array named `kernel` of 1s and a central 0 (indicating what surrounding cells to count),  `convolve2d()`, together with a boolean array will give us a count of surrounding `True` values for each cell. So if lumberyards are represented by the integer value `3`, then we can get a count of adjacent lumberyards with:

```python
>>> kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
>>> print(forest)  # custom class, ._matrix is the numpy array
.#|#
|||#
#|#|
...#
>>> convolve2d(forest._matrix == 3, kernel, mode='same')
array([[1, 0, 3, 1],
       [2, 3, 4, 2],
       [0, 2, 2, 3],
       [1, 2, 2, 1]])
```

We have three rules, each applying to one of the three states, and all 3 are simple counts of the surrounding area. So you can make the following assignments:

- *trees*: if *open* and *surrounding tree count* is three or more
- *lumberyard*: if *trees* and *surrounding lumberyard count* is three or more
- *open*: if *lumberyard* and *surrounding lumberyard count* is zero *or* *surrounding trees count* is zero

So for each cell, we need three neighbor counts for each of the types.

We can cast those rules as broadcasted numpy tests, then use `np.select()` with those tests to select either the replacement value or the original input cell:

```python
output = np.select(
    [trees_cond, lumberyard_cond, open_cond],
    [trees, lumberyards, opens],
    default=forest.matrix
)
```

In [1]:
from enum import Enum
from typing import Callable, Dict, Optional, Sequence, Tuple
from scipy.signal import convolve2d
import numpy as np

class Acre(Enum):
    open = 1, '.'
    trees = 2, '|'
    lumberyard = 3, '#'
    
    def __new__(cls, int, value):
        instance = object.__new__(cls)
        instance._value_ = value
        instance.int = int
        return instance

_kernel = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
# called for each step
_animation_callback = Callable[[np.ndarray], None]
    
class Forest:
    def __init__(self, forestmap: str) -> None:
        self._matrix = np.array([
            Acre(c).int for line in forestmap.splitlines() for c in line
        ]).reshape((forestmap.index('\n'), -1))

    def __str__(self) -> str:
        mapping = {a.int: a.value for a in Acre}
        return '\n'.join([''.join(map(mapping.__getitem__, row)) for row in self._matrix])
    
    @property
    def total_resource_value(self):
        counts = dict(zip(*np.unique(self._matrix, return_counts=True)))
        return counts[Acre.trees.int] * counts[Acre.lumberyard.int]        

    def run(self, minutes: int, _callback: Optional[_animation_callback] = None) -> int:
        """Run for the given amount of time, then return the resource value"""
        f = self._matrix
        full = {a: np.full(f.shape, a.int) for a in Acre}
        # map state (matrix flattened to a tuple) to the minute we saw this state
        # this lets us jump ahead to a future state once a repeating pattern sets in.
        # When we see this, repeat_start is set to the point in time the repetitions
        # started, and repeated_states is the repeated pattern sequence as tuples
        seen: Dict[Tuple, int] = {}
        repeat_start: Optional[int] = None
        repeated_states: Sequence[Tuple[int]] = []
        for m in range(minutes):
            counts = {acre: convolve2d(f == acre.int, _kernel, mode='same') for acre in Acre}
            rules = {
                Acre.trees: (  # currently open, and has 3 or more trees as neighbours
                    (f == Acre.open.int) & (counts[Acre.trees] >= 3)
                ),
                Acre.lumberyard: (  # currently trees, and has 3 or more lumberyards as neighbours
                    (f == Acre.trees.int) & (counts[Acre.lumberyard] >= 3)
                ),
                Acre.open: (  # currently lumberyard, but either missing neighbouring lumberyards or trees
                    (f == Acre.lumberyard.int) & (
                        (counts[Acre.lumberyard] == 0) | (counts[Acre.trees] == 0)
                    )
                ),
            }
            f = np.select(list(rules.values()), [full[a] for a in rules], default=f)
            if _callback:
                _callback(f)
            key = tuple(f.flatten())
            if key in seen:
                repeat_start = seen[key]
                _selected = ((t, s) for s, t in seen.items() if repeat_start <= t < m)
                repeated_states = [s for _, s in sorted(_selected)]
                break
            seen[key] = m
                
        if repeat_start is not None:
            # jump forward in time
            state = repeated_states[(minutes - (repeat_start + 1)) % len(repeated_states)]
            f = np.array(state).reshape(f.shape)
        self._matrix = f

In [2]:
testforest = Forest('''\
.#.#...|#.
.....#|##|
.|..|...#.
..|#.....#
#.#|||#|#|
...#.||...
.|....|...
||...#|.#|
|.||||..|.
...#.|..|.''')
testforest.run(10)
assert testforest.total_resource_value == 1147

In [3]:
import aocd

data = aocd.get_data(day=18, year=2018)

In [4]:
forest = Forest(data)
forest.run(10)
print('Part 1:', forest.total_resource_value)

Part 1: 564375


## Part 2 - scaling this up

While numpy makes this really fast, running this a billion times is still going to take way too much time, so we look for a shortcut again. The forrest stabilises and produces a looping pattern after 500 steps or so, so we can extrapolate from there.

We need the periodicity of the loop so we can figure out how many steps beyond the point we detect the loop we need to go before we have the same state as minute 1 billion will have. Given the minute the pattern started repeating $\rho$, and $N$ repeating states, we can use the $n$th repeating state calculated with:

$$n = 1.000.000.000 - \rho \pmod N)$$

but take into account that in Python, the minute counter starts at 0, not 1.

In [5]:
forest = Forest(data)
forest.run(1000)
print('Part 2:', forest.total_resource_value)

Part 2: 189720


The animation produced below can be viewed online [via the Jupyter notebook viewer](https://nbviewer.jupyter.org/github/mjpieters/adventofcode/blob/master/2018/Day%2018.ipynb); the GitHub renderer filters the video out.

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import animation
plt.rc('animation', html='html5')

def animate(forest):
    fig, ax = plt.subplots()
    fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
    ax.set_axis_off()
    
    # colours for open, trees and lumberyards 
    cmap = ListedColormap(['greenyellow', 'forestgreen', 'saddlebrown'])
    
    frames = []
    def create_frame(m):
        frames.append([plt.imshow(m, cmap=cmap, animated=True)])
    forest.run(600, create_frame)
    
    anim = animation.ArtistAnimation(
        fig, frames, interval=100, blit=True,
        repeat_delay=1000
    )
    plt.close(fig)
    return anim

animate(Forest(data))